# [Advent of Code 2022 Day ?]()

?

## Initial setup

In [1]:
import ipytest
import pytest
import sys
sys.path.append("..")
from ansi import *
from comp import *
ipytest.autoconfig()
PART_ONE_SENTINEL = 0x3f3f3f3f + 1
PART_TWO_SENTINEL = 0x3f3f3f3f + 2

## Input Parsing

In [2]:
def parse_input(filename: str) -> dict[str, Any]:

    gen = yield_line(filename)

    context: dict[str, Any] = {
        "input": [],
    }

    for line in gen:
        context["input"].append(parse(r"([UDLR]) (\d+)", line))

    return context

## Sandbox

In [3]:
def adja(head, tail):
    hx, hy = head
    tx, ty = tail

    if head == tail:
        return True

    for dx, dy in DIR.ADJA:
        if ((hx + dx), (hy + dy)) == tail:
            return True

    return False

def surr(head, tail):
    hx, hy = head
    tx, ty = tail

    if head == tail:
        return True

    for dx, dy in DIR.SURR:
        if ((hx + dx), (hy + dy)) == tail:
            return True

    return False

def diagonal(head, tail):
    hx, hy = head
    tx, ty = tail

    if (hx + 1, hy + 1) == tail:
        return True

    if (hx - 1, hy - 1) == tail:
        return True

    if (hx + 1, hy - 1) == tail:
        return True

    if (hx + 1, hy + 1) == tail:
        return True

def same_rank(head, tail):
    return head[0] == tail[0] or head[1] == tail[1]

heading = {
    "U": (-1, 0),
    "D": (+1, 0),
    "L": (0, -1),
    "R": (0, +1),
}

def get_next(context, head, tail, direction, magnitude):

    hx, hy = head
    tx, ty = tail
    dx, dy = heading[direction]

    for i in range(magnitude):
        print(f"round {i + 1} {(hx, hy)} {(tx, ty)}")
        assert surr((hx, hy), (tx, ty))
        hx += dx
        hy += dy
        context["visited"].add((tx, ty))
        if surr((hx, hy), (tx, ty)):
            continue
        else:
            if same_rank((hx, hy), (tx, ty)):
                for a, b in DIR.ADJA:
                    if adja((hx, hy), (tx + a, ty + b)):
                        context["visited"].add((tx + a, ty + b))
                        tx += a
                        ty += b
                        break
                else:
                    raise Exception("Failed to find something")
            else:
                for a, b in DIR.DIAG:
                    if adja((hx, hy), (tx + a, ty + b)):
                        context["visited"].add((tx + a, ty + b))
                        tx += a
                        ty += b
                        break
                else:
                    raise Exception("Failed to find something for part 2")


    return (hx, hy), (tx, ty)

def draw(dots):
    board = []
    for _ in range(30):
        board.append(["."] * 30)

    for i in reversed(range(len(dots))):
        x, y = dots[i]
        if i == 0:
            char = "H"
        else:
            char = str(int(i) + 0)
        board[x + 10][y + 10] = char

    for b in board:
        print("".join(b))
    print()

def process(part: int, context: dict[str, Any]) -> int:
    if part == 1:
        context["visited"] = set()

        head = (0, 0)
        tail = (0, 0)

        print(head, tail)

        for direction, magnitude in context["input"]:
            head, tail = get_next(context, head, tail, direction, int(magnitude))
            print(head, tail)

        return len(context["visited"])
    if part == 2:
        print(context["input"])
        context["visited"] = {(0, 0)}
        dots = [
            [0, 0], # idx 0 is you
            [0, 0],
            [0, 0],
            [0, 0],
            [0, 0],
            [0, 0],
            [0, 0],
            [0, 0],
            [0, 0],
            [0, 0], # idx 9 is 9
        ]

        def update(index: int, new_x: int, new_y: int) -> bool:
            if index == 9:
                return True
            tx, ty = dots[index + 1]
            if surr((new_x, new_y), (tx, ty)):
                return True
            if same_rank((new_x, new_y), (tx, ty)):
                for dx, dy in DIR.ADJA:
                    dots[index + 1][0] += dx
                    dots[index + 1][1] += dy
                    if surr((new_x, new_y), (dots[index + 1][0], dots[index + 1][1])) and update(index + 1, dots[index + 1][0], dots[index + 1][1]):
                        return True
                    dots[index + 1][0] -= dx
                    dots[index + 1][1] -= dy
            else:
                for dx, dy in DIR.DIAG:
                    dots[index + 1][0] += dx
                    dots[index + 1][1] += dy
                    if surr((new_x, new_y), (dots[index + 1][0], dots[index + 1][1])) and update(index + 1, dots[index + 1][0], dots[index + 1][1]):
                        return True
                    dots[index + 1][0] -= dx
                    dots[index + 1][1] -= dy
            return False

        for direction, magnitude in context["input"]:
            for i in range(int(magnitude)):
                x, y = dots[0]
                dx, dy = heading[direction]
                dots[0] = x + dx, y + dy
                update(0, x + dx, y + dy)
                context["visited"].add((dots[9][0], dots[9][1]))

        return len(context["visited"])
    else:
        raise Exception(f"Invalid part: {part}")

## Part 1
Lorem ipsum

In [4]:
def part_one(context: dict[str, Any]) -> int:
    return process(1, context)

## Part 1 Testing

In [5]:
PART_ONE_CASES: dict[str, str | int] = {
    "example1": 13,
    # "example2": PART_ONE_SENTINEL,
    # "example3": PART_ONE_SENTINEL,
    # "example4": PART_ONE_SENTINEL,
    # "example5": PART_ONE_SENTINEL,
    # "example6": PART_ONE_SENTINEL,
    # "example7": PART_ONE_SENTINEL,
    # "example8": PART_ONE_SENTINEL,
    # "example9": PART_ONE_SENTINEL,
    "input": 5874,
}
PART_ONE_OUTPUTS: dict[str, str | int] = dict()

In [6]:
%%ipytest

def test_surr():
    assert surr((0, 0), (1, 1))
    assert surr((0, 0), (-1, -1))
    assert diagonal((0, 0), (1, 1))
    assert diagonal((0, 0), (-1, -1))
    assert not adja((0, 0), (1, 1))
    assert not adja((0, 0), (-1, -1))

@pytest.mark.parametrize("test_file_name, test_expected_output", PART_ONE_CASES.items())
def test_part_one(test_file_name, test_expected_output):
    test_actual_output = part_one(parse_input(test_file_name))
    PART_ONE_OUTPUTS[test_file_name] = test_actual_output
    assert test_actual_output == test_expected_output

...                                                                                          [100%]
3 passed in 0.27s


## Part 2
Lorem ipsum

In [7]:
def part_two(context: dict[str, Any]) -> int:
    return process(2, context)

## Part 2 Testing

In [8]:
PART_TWO_CASES: dict[str, str | int] = {
    # "example1": 36,
    "example2": 36,
    # "example3": PART_TWO_SENTINEL,
    # "example4": PART_TWO_SENTINEL,
    # "example5": PART_TWO_SENTINEL,
    # "example6": PART_TWO_SENTINEL,
    # "example7": PART_TWO_SENTINEL,
    # "example8": PART_TWO_SENTINEL,
    # "example9": PART_TWO_SENTINEL,
    "input": 2467,
}
PART_TWO_OUTPUTS: dict[str, str | int] = dict()

In [9]:
%%ipytest
@pytest.mark.parametrize("test_file_name, test_expected_output", PART_TWO_CASES.items())
def test_part_two(test_file_name, test_expected_output):  # not 6288 or 4852
    test_actual_output = part_two(parse_input(test_file_name))
    PART_TWO_OUTPUTS[test_file_name] = test_actual_output
    assert test_actual_output == test_expected_output

..                                                                                           [100%]
2 passed in 0.43s


In [10]:
for file_name, actual_output in PART_TWO_OUTPUTS.items():
    expected = PART_TWO_CASES[file_name]
    actual = actual_output
    outcome = ("SENTINEL" if actual == expected else "CANDIDATE") if expected == PART_TWO_SENTINEL else ("CORRECT" if expected == actual else "INCORRECT")
    print(f"{file_name:>10}      expected: {expected:>10}      actual: {actual:>10}      outcome: {outcome}")

  example2      expected:         36      actual:         36      outcome: CORRECT
     input      expected:       2467      actual:       2467      outcome: CORRECT
